# Sistema de Gestión de Empleados con Árbol B+

## Materia: Estructuras de Datos

Este notebook implementa un sistema completo de base de datos de empleados utilizando **Árbol B+** como estructura central.

### ¿Por qué Árbol B+?

El Árbol B+ es la estructura de datos fundamental usada en bases de datos reales (MySQL, PostgreSQL, Oracle, etc.) porque:

- **Balanceo automático**: Siempre mantiene O(log n) para búsquedas
- **Optimizado para disco**: Minimiza accesos a disco
- **Datos en hojas**: Todos los registros están en las hojas
- **Hojas enlazadas**: Recorrido secuencial eficiente O(n)
- **Búsqueda por rango**: Muy eficiente
- **Alto factor de ramificación**: Árbol más ancho y menos alto

### Características del sistema:
- Tabla de 4 columnas: nombre, edad, sueldo, cargo
- Índices configurables en cualquier columna usando Árbol B+
- Carga de datos desde archivo CSV
- Entrada manual de datos
- Búsquedas eficientes: exacta, rango, ordenada

## 1. Importar Librerías

In [ ]:
import csv
import io
from typing import Any, List, Dict, Optional, Tuple
from dataclasses import dataclass
import time
import math

## 2. Estructura de Datos: Empleado

In [ ]:
@dataclass
class Employee:
    """Clase que representa un empleado (registro de la tabla)"""
    nombre: str
    edad: int
    sueldo: float
    cargo: str
    id: int = 0  # ID único del empleado en la tabla
    
    def __repr__(self):
        return f"Employee(#{self.id}: {self.nombre}, {self.edad}, ${self.sueldo:,.2f}, {self.cargo})"
    
    def to_dict(self):
        return {
            'id': self.id,
            'nombre': self.nombre,
            'edad': self.edad,
            'sueldo': self.sueldo,
            'cargo': self.cargo
        }
    
    def get_field(self, field_name: str):
        """Obtener valor de un campo"""
        return getattr(self, field_name)


print("✓ Clase Employee definida")

## 3. Implementación del Árbol B+

### 3.1 Nodo del Árbol B+

Un Árbol B+ tiene dos tipos de nodos:
- **Nodos internos**: Solo contienen claves para navegación
- **Nodos hoja**: Contienen claves y punteros a los datos reales

In [ ]:
class BPlusTreeNode:
    """Nodo del Árbol B+"""
    
    def __init__(self, order: int, is_leaf: bool = False):
        """
        Args:
            order: Orden del árbol (número máximo de hijos)
            is_leaf: True si es nodo hoja
        """
        self.order = order
        self.is_leaf = is_leaf
        self.keys = []  # Claves del nodo
        self.children = []  # Hijos (para nodos internos) o datos (para hojas)
        self.next = None  # Puntero al siguiente nodo hoja (solo para hojas)
        self.parent = None  # Puntero al padre
    
    def is_full(self) -> bool:
        """Verificar si el nodo está lleno"""
        return len(self.keys) >= self.order - 1
    
    def is_underflow(self) -> bool:
        """Verificar si el nodo tiene muy pocas claves"""
        min_keys = math.ceil(self.order / 2) - 1
        return len(self.keys) < min_keys
    
    def __repr__(self):
        type_str = "LEAF" if self.is_leaf else "INTERNAL"
        return f"BPlusNode({type_str}, keys={self.keys[:5]}{'...' if len(self.keys) > 5 else ''})"


print("✓ Clase BPlusTreeNode implementada")

### 3.2 Árbol B+ Completo

In [ ]:
class BPlusTree:
    """
    Árbol B+ para indexación de base de datos
    
    Propiedades:
    - Todos los datos están en las hojas
    - Hojas enlazadas para recorrido secuencial
    - Balanceo automático
    - Búsqueda, inserción, eliminación en O(log n)
    """
    
    def __init__(self, order: int = 4):
        """
        Args:
            order: Orden del árbol (número máximo de hijos por nodo)
                   Típicamente entre 3 y 100+ para bases de datos reales
        """
        if order < 3:
            raise ValueError("El orden debe ser al menos 3")
        
        self.order = order
        self.root = BPlusTreeNode(order, is_leaf=True)
        self.min_keys = math.ceil(order / 2) - 1
    
    def search(self, key) -> List[int]:
        """
        Buscar una clave en el árbol
        
        Returns:
            Lista de IDs de empleados con esa clave
        """
        leaf = self._find_leaf(key)
        
        # Buscar la clave en el nodo hoja
        for i, k in enumerate(leaf.keys):
            if k == key:
                return leaf.children[i]  # Retorna lista de IDs
        
        return []
    
    def _find_leaf(self, key) -> BPlusTreeNode:
        """Encontrar el nodo hoja que debería contener la clave"""
        node = self.root
        
        # Navegar hasta la hoja
        while not node.is_leaf:
            # Encontrar el índice del hijo correcto
            i = 0
            while i < len(node.keys) and key >= node.keys[i]:
                i += 1
            node = node.children[i]
        
        return node
    
    def insert(self, key, emp_id: int):
        """Insertar una clave con su ID de empleado"""
        leaf = self._find_leaf(key)
        
        # Buscar si la clave ya existe
        for i, k in enumerate(leaf.keys):
            if k == key:
                # Clave existe, agregar ID a la lista
                if emp_id not in leaf.children[i]:
                    leaf.children[i].append(emp_id)
                return
        
        # Insertar nueva clave en orden
        i = 0
        while i < len(leaf.keys) and key > leaf.keys[i]:
            i += 1
        
        leaf.keys.insert(i, key)
        leaf.children.insert(i, [emp_id])
        
        # Si el nodo está lleno, dividir
        if leaf.is_full():
            self._split_leaf(leaf)
    
    def _split_leaf(self, leaf: BPlusTreeNode):
        """Dividir un nodo hoja lleno"""
        mid = len(leaf.keys) // 2
        
        # Crear nuevo nodo hoja
        new_leaf = BPlusTreeNode(self.order, is_leaf=True)
        new_leaf.keys = leaf.keys[mid:]
        new_leaf.children = leaf.children[mid:]
        
        # Actualizar hojas enlazadas
        new_leaf.next = leaf.next
        leaf.next = new_leaf
        
        # Actualizar hoja original
        leaf.keys = leaf.keys[:mid]
        leaf.children = leaf.children[:mid]
        
        # Promover clave al padre
        promote_key = new_leaf.keys[0]
        
        if leaf == self.root:
            # Crear nueva raíz
            new_root = BPlusTreeNode(self.order, is_leaf=False)
            new_root.keys = [promote_key]
            new_root.children = [leaf, new_leaf]
            leaf.parent = new_root
            new_leaf.parent = new_root
            self.root = new_root
        else:
            # Insertar en padre existente
            self._insert_in_parent(leaf, promote_key, new_leaf)
    
    def _insert_in_parent(self, left: BPlusTreeNode, key, right: BPlusTreeNode):
        """Insertar clave en nodo padre"""
        parent = left.parent
        
        # Encontrar posición de inserción
        i = 0
        while i < len(parent.keys) and key > parent.keys[i]:
            i += 1
        
        parent.keys.insert(i, key)
        parent.children.insert(i + 1, right)
        right.parent = parent
        
        # Si el padre está lleno, dividir
        if parent.is_full():
            self._split_internal(parent)
    
    def _split_internal(self, node: BPlusTreeNode):
        """Dividir un nodo interno lleno"""
        mid = len(node.keys) // 2
        promote_key = node.keys[mid]
        
        # Crear nuevo nodo interno
        new_node = BPlusTreeNode(self.order, is_leaf=False)
        new_node.keys = node.keys[mid + 1:]
        new_node.children = node.children[mid + 1:]
        
        # Actualizar padres de los hijos
        for child in new_node.children:
            child.parent = new_node
        
        # Actualizar nodo original
        node.keys = node.keys[:mid]
        node.children = node.children[:mid + 1]
        
        if node == self.root:
            # Crear nueva raíz
            new_root = BPlusTreeNode(self.order, is_leaf=False)
            new_root.keys = [promote_key]
            new_root.children = [node, new_node]
            node.parent = new_root
            new_node.parent = new_root
            self.root = new_root
        else:
            # Insertar en padre existente
            self._insert_in_parent(node, promote_key, new_node)
    
    def range_search(self, min_key, max_key) -> List[int]:
        """
        Buscar todas las claves en el rango [min_key, max_key]
        
        Esta es una de las ventajas principales del Árbol B+:
        búsqueda por rango muy eficiente usando las hojas enlazadas
        """
        result = []
        
        # Encontrar la primera hoja que contiene min_key
        leaf = self._find_leaf(min_key)
        
        # Recorrer las hojas enlazadas
        while leaf:
            for i, key in enumerate(leaf.keys):
                if min_key <= key <= max_key:
                    result.extend(leaf.children[i])
                elif key > max_key:
                    return result
            
            leaf = leaf.next
        
        return result
    
    def get_all_sorted(self) -> List[Tuple[Any, List[int]]]:
        """
        Obtener todos los pares (clave, IDs) en orden
        
        Ventaja del Árbol B+: recorrido secuencial O(n) muy eficiente
        simplemente siguiendo los enlaces de las hojas
        """
        result = []
        
        # Encontrar la primera hoja (más a la izquierda)
        leaf = self.root
        while not leaf.is_leaf:
            leaf = leaf.children[0]
        
        # Recorrer todas las hojas enlazadas
        while leaf:
            for i, key in enumerate(leaf.keys):
                result.append((key, leaf.children[i]))
            leaf = leaf.next
        
        return result
    
    def get_height(self) -> int:
        """Obtener altura del árbol"""
        height = 0
        node = self.root
        while not node.is_leaf:
            height += 1
            node = node.children[0]
        return height + 1
    
    def get_stats(self) -> Dict:
        """Obtener estadísticas del árbol"""
        num_keys = 0
        num_leaves = 0
        num_internal = 0
        
        # Contar nodos hoja
        leaf = self.root
        while not leaf.is_leaf:
            leaf = leaf.children[0]
        
        while leaf:
            num_leaves += 1
            num_keys += len(leaf.keys)
            leaf = leaf.next
        
        # Contar nodos internos (BFS)
        if not self.root.is_leaf:
            queue = [self.root]
            while queue:
                node = queue.pop(0)
                if not node.is_leaf:
                    num_internal += 1
                    queue.extend(node.children)
        
        return {
            'height': self.get_height(),
            'order': self.order,
            'num_keys': num_keys,
            'num_leaves': num_leaves,
            'num_internal': num_internal,
            'total_nodes': num_leaves + num_internal
        }


print("✓ Clase BPlusTree implementada completamente")
print("  - Búsqueda: O(log n)")
print("  - Inserción: O(log n)")
print("  - Búsqueda por rango: O(log n + k) donde k = resultados")
print("  - Recorrido ordenado: O(n) usando hojas enlazadas")

## 4. Clase Principal: EmployeeTable

### Sistema de base de datos de empleados usando Árbol B+ para índices

In [ ]:
class EmployeeTable:
    """
    Tabla de empleados con índices usando Árbol B+
    
    Similar a cómo funcionan las bases de datos reales:
    - Almacenamiento principal: lista de empleados
    - Índices: Árbol B+ que mapea valores de columna → IDs de empleados
    """
    
    def __init__(self, bplus_order: int = 4):
        """
        Args:
            bplus_order: Orden del Árbol B+ (típicamente 3-100+)
        """
        self.employees = []  # Almacenamiento principal (lista de Employee)
        self.indices = {}  # {columna: BPlusTree}
        self.bplus_order = bplus_order
        self.next_id = 0
    
    def add_employee(self, nombre: str, edad: int, sueldo: float, cargo: str) -> int:
        """Agregar empleado a la tabla"""
        if cargo not in ['empleado', 'jefe', 'propietario']:
            raise ValueError(f"Cargo inválido: {cargo}. Debe ser: empleado, jefe, o propietario")
        
        # Crear empleado con ID único
        employee = Employee(nombre, edad, sueldo, cargo, id=self.next_id)
        self.employees.append(employee)
        emp_id = self.next_id
        self.next_id += 1
        
        # Actualizar todos los índices existentes
        for column, bplus_tree in self.indices.items():
            value = employee.get_field(column)
            bplus_tree.insert(value, emp_id)
        
        return emp_id
    
    def create_index(self, column: str):
        """
        Crear índice Árbol B+ en una columna específica
        
        Similar a: CREATE INDEX idx_edad ON empleados(edad);
        """
        if column not in ['nombre', 'edad', 'sueldo', 'cargo']:
            raise ValueError(f"Columna inválida: {column}")
        
        if column in self.indices:
            print(f"⚠ La columna '{column}' ya tiene un índice. Reconstruyendo...")
        
        # Crear nuevo Árbol B+
        bplus_tree = BPlusTree(order=self.bplus_order)
        
        # Construir índice insertando todos los empleados
        for emp in self.employees:
            value = emp.get_field(column)
            bplus_tree.insert(value, emp.id)
        
        self.indices[column] = bplus_tree
        
        # Mostrar estadísticas
        stats = bplus_tree.get_stats()
        print(f"\n✓ Índice Árbol B+ creado para la columna '{column}'")
        print(f"  - Orden: {stats['order']}")
        print(f"  - Altura: {stats['height']}")
        print(f"  - Claves únicas: {stats['num_keys']}")
        print(f"  - Nodos hoja: {stats['num_leaves']}")
        print(f"  - Nodos internos: {stats['num_internal']}")
        print(f"  - Total nodos: {stats['total_nodes']}")
    
    def search(self, column: str, value) -> List[Employee]:
        """
        Buscar empleados por valor en una columna
        
        Similar a: SELECT * FROM empleados WHERE {column} = {value};
        """
        if column in self.indices:
            # Usar índice Árbol B+ - O(log n)
            bplus_tree = self.indices[column]
            emp_ids = bplus_tree.search(value)
            return [self.employees[emp_id] for emp_id in emp_ids]
        else:
            # Búsqueda lineal - O(n)
            result = []
            for emp in self.employees:
                if emp.get_field(column) == value:
                    result.append(emp)
            return result
    
    def range_search(self, column: str, min_value, max_value) -> List[Employee]:
        """
        Buscar empleados en un rango de valores
        
        Similar a: SELECT * FROM empleados WHERE {column} BETWEEN {min} AND {max};
        
        El Árbol B+ es EXCELENTE para esto: O(log n + k)
        """
        if column in self.indices:
            # Usar índice Árbol B+ - O(log n + k)
            bplus_tree = self.indices[column]
            emp_ids = bplus_tree.range_search(min_value, max_value)
            return [self.employees[emp_id] for emp_id in emp_ids]
        else:
            # Búsqueda lineal - O(n)
            result = []
            for emp in self.employees:
                value = emp.get_field(column)
                if min_value <= value <= max_value:
                    result.append(emp)
            return result
    
    def get_all_sorted(self, column: str) -> List[Employee]:
        """
        Obtener todos los empleados ordenados por una columna
        
        Similar a: SELECT * FROM empleados ORDER BY {column};
        
        Con Árbol B+ es muy eficiente: O(n) usando hojas enlazadas
        """
        if column in self.indices:
            # Usar índice Árbol B+ - O(n) recorrido secuencial
            bplus_tree = self.indices[column]
            sorted_pairs = bplus_tree.get_all_sorted()
            result = []
            for key, emp_ids in sorted_pairs:
                for emp_id in emp_ids:
                    result.append(self.employees[emp_id])
            return result
        else:
            # Ordenar - O(n log n)
            return sorted(self.employees, key=lambda e: e.get_field(column))
    
    def load_from_csv(self, csv_content: str):
        """Cargar empleados desde contenido CSV"""
        reader = csv.DictReader(io.StringIO(csv_content))
        count = 0
        
        for row in reader:
            try:
                nombre = row['nombre'].strip()
                edad = int(row['edad'])
                sueldo = float(row['sueldo'])
                cargo = row['cargo'].strip().lower()
                
                self.add_employee(nombre, edad, sueldo, cargo)
                count += 1
            except (KeyError, ValueError) as e:
                print(f"⚠ Error en fila {count + 1}: {e}")
                continue
        
        print(f"\n✓ {count} empleados cargados desde CSV")
    
    def load_from_file(self, filename: str):
        """Cargar empleados desde archivo CSV"""
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                content = f.read()
            self.load_from_csv(content)
        except FileNotFoundError:
            print(f"✗ Archivo no encontrado: {filename}")
        except Exception as e:
            print(f"✗ Error al cargar archivo: {e}")
    
    def show_all(self):
        """Mostrar todos los empleados"""
        if not self.employees:
            print("No hay empleados en la tabla.")
            return
        
        print("\n" + "=" * 80)
        print(f"{'ID':<5} {'Nombre':<25} {'Edad':<6} {'Sueldo':<15} {'Cargo':<15}")
        print("=" * 80)
        
        for emp in self.employees:
            print(f"{emp.id:<5} {emp.nombre:<25} {emp.edad:<6} ${emp.sueldo:<14,.2f} {emp.cargo:<15}")
        
        print("=" * 80)
        print(f"Total: {len(self.employees)} empleados\n")
    
    def show_index_stats(self):
        """Mostrar estadísticas de los índices"""
        if not self.indices:
            print("\nNo hay índices creados.")
            print("Usa create_index('columna') para crear un índice.")
            return
        
        print("\n" + "=" * 70)
        print("ESTADÍSTICAS DE ÍNDICES (Árbol B+)")
        print("=" * 70)
        
        for column, bplus_tree in self.indices.items():
            stats = bplus_tree.get_stats()
            print(f"\n📊 Columna: {column}")
            print(f"  - Orden del árbol: {stats['order']}")
            print(f"  - Altura: {stats['height']}")
            print(f"  - Claves únicas: {stats['num_keys']}")
            print(f"  - Nodos hoja: {stats['num_leaves']}")
            print(f"  - Nodos internos: {stats['num_internal']}")
            print(f"  - Total nodos: {stats['total_nodes']}")
            
            # Mostrar primeros valores
            sorted_pairs = bplus_tree.get_all_sorted()[:5]
            if sorted_pairs:
                print(f"  - Primeros valores:")
                for key, emp_ids in sorted_pairs:
                    print(f"    • {key}: {len(emp_ids)} empleado(s)")
        
        print("\n" + "=" * 70)
    
    def get_count(self) -> int:
        """Obtener número de empleados"""
        return len(self.employees)


print("✓ Clase EmployeeTable implementada")
print("  Usa Árbol B+ para todos los índices")

## 5. Funciones Auxiliares

In [ ]:
def create_sample_csv():
    """Crear contenido CSV de ejemplo"""
    return """nombre,edad,sueldo,cargo
Juan Pérez,28,45000,empleado
María García,35,75000,jefe
Carlos López,28,48000,empleado
Ana Martínez,42,150000,propietario
Pedro Sánchez,31,52000,empleado
Laura Rodríguez,35,78000,jefe
Miguel Torres,28,46000,empleado
Sofia Ramírez,39,85000,jefe
Diego Flores,25,42000,empleado
Elena Castro,45,200000,propietario
Roberto Jiménez,28,47000,empleado
Carmen Vega,50,180000,propietario
Fernando Díaz,33,55000,empleado
Isabel Moreno,38,82000,jefe
Luis Herrera,29,49000,empleado
Patricia Gómez,26,44000,empleado
Ricardo Núñez,40,90000,jefe
Verónica Silva,28,47500,empleado
Andrés Castro,32,54000,empleado
Claudia Vargas,36,76000,jefe"""


def save_sample_csv(filename='empleados.csv'):
    """Guardar CSV de ejemplo en archivo"""
    content = create_sample_csv()
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"✓ Archivo '{filename}' creado con datos de ejemplo")


def print_results(results: List[Employee], title: str):
    """Imprimir resultados de búsqueda"""
    print(f"\n{title}")
    print("-" * 75)
    if results:
        print(f"Encontrados {len(results)} resultado(s):\n")
        for emp in results:
            print(f"  • #{emp.id:<3} {emp.nombre:<25} - {emp.edad} años - ${emp.sueldo:,.2f} - {emp.cargo}")
    else:
        print("No se encontraron resultados.")
    print()


def compare_performance(table: EmployeeTable, column: str, value):
    """Comparar rendimiento con y sin índice"""
    print(f"\n{'='*70}")
    print(f"COMPARACIÓN DE RENDIMIENTO: {column} = {value}")
    print(f"{'='*70}")
    
    # Búsqueda con índice B+
    if column in table.indices:
        start = time.perf_counter()
        results = table.search(column, value)
        end = time.perf_counter()
        time_with_index = (end - start) * 1000000  # en microsegundos
        print(f"✓ Con índice B+: {time_with_index:.2f} μs ({len(results)} resultados)")
        print(f"  Complejidad: O(log n)")
    
    # Búsqueda lineal (sin índice)
    start = time.perf_counter()
    results_linear = []
    for emp in table.employees:
        if emp.get_field(column) == value:
            results_linear.append(emp)
    end = time.perf_counter()
    time_linear = (end - start) * 1000000  # en microsegundos
    print(f"✗ Sin índice (lineal): {time_linear:.2f} μs ({len(results_linear)} resultados)")
    print(f"  Complejidad: O(n)")
    
    if column in table.indices and time_linear > 0 and time_with_index > 0:
        speedup = time_linear / time_with_index
        print(f"\n⚡ Aceleración: {speedup:.2f}x más rápido con índice B+")
    
    print(f"{'='*70}\n")


print("✓ Funciones auxiliares definidas")

## 6. Ejemplos de Uso

### 6.1 Crear tabla y cargar datos desde CSV

In [ ]:
# Crear tabla con Árbol B+ de orden 4
# (En bases de datos reales, el orden suele ser 50-200)
tabla = EmployeeTable(bplus_order=4)

# Cargar datos desde CSV
print("Cargando empleados desde CSV...")
csv_content = create_sample_csv()
tabla.load_from_csv(csv_content)

# Mostrar todos
tabla.show_all()

### 6.2 Crear índice en columna 'edad'

In [ ]:
# Crear índice Árbol B+ en edad
# Similar a: CREATE INDEX idx_edad ON empleados(edad);
tabla.create_index('edad')

# Buscar empleados de 28 años - O(log n)
results = tabla.search('edad', 28)
print_results(results, "EMPLEADOS DE 28 AÑOS (usando índice B+)")

### 6.3 Búsqueda por rango (ventaja del B+)

In [ ]:
# Búsqueda por rango - O(log n + k) con B+
# Similar a: SELECT * FROM empleados WHERE edad BETWEEN 30 AND 40;
results_range = tabla.range_search('edad', 30, 40)
print_results(results_range, "EMPLEADOS ENTRE 30 Y 40 AÑOS (búsqueda por rango con B+)")

### 6.4 Obtener datos ordenados (otra ventaja del B+)

In [ ]:
# Obtener todos ordenados por edad - O(n) con B+
# Similar a: SELECT * FROM empleados ORDER BY edad;
sorted_emps = tabla.get_all_sorted('edad')
print_results(sorted_emps[:10], "PRIMEROS 10 EMPLEADOS ORDENADOS POR EDAD (usando B+)")

### 6.5 Crear más índices

In [ ]:
# Crear índice en cargo
tabla.create_index('cargo')

# Buscar todos los jefes
results = tabla.search('cargo', 'jefe')
print_results(results, "TODOS LOS JEFES (usando índice B+)")

# Crear índice en sueldo
tabla.create_index('sueldo')

# Buscar sueldos en rango
results_salary = tabla.range_search('sueldo', 70000, 100000)
print_results(results_salary, "EMPLEADOS CON SUELDO ENTRE $70,000 Y $100,000")

### 6.6 Ver estadísticas de los índices B+

In [ ]:
# Mostrar estadísticas detalladas
tabla.show_index_stats()

### 6.7 Comparación de rendimiento

In [ ]:
# Comparar búsqueda con B+ vs lineal
compare_performance(tabla, 'edad', 28)
compare_performance(tabla, 'cargo', 'jefe')
compare_performance(tabla, 'sueldo', 75000)

### 6.8 Agregar más empleados y ver actualización automática

In [ ]:
# Agregar nuevos empleados
print("\nAgregando nuevos empleados...\n")
tabla.add_employee("Gabriela Torres", 28, 46800.00, "empleado")
tabla.add_employee("Sebastián Rojas", 28, 47200.00, "empleado")
tabla.add_employee("Valentina Méndez", 34, 68000.00, "jefe")

# Los índices B+ se actualizan automáticamente
results = tabla.search('edad', 28)
print_results(results, "EMPLEADOS DE 28 AÑOS (después de agregar nuevos)")

print(f"\nTotal de empleados en la tabla: {tabla.get_count()}")

# Ver estadísticas actualizadas
tabla.show_index_stats()

## 7. Ejemplo Interactivo - Crea tu propia tabla

In [ ]:
# Crea tu propio CSV (modifica los datos)
mi_csv = """nombre,edad,sueldo,cargo
Tu Nombre,25,50000,empleado
Otro Nombre,30,75000,jefe
Más Nombres,35,120000,propietario
"""

# Crear tu tabla
mi_tabla = EmployeeTable(bplus_order=3)  # Orden pequeño para ver estructura
mi_tabla.load_from_csv(mi_csv)
mi_tabla.show_all()

# Crear índices
mi_tabla.create_index('edad')
mi_tabla.create_index('cargo')

# Hacer búsquedas
# resultados = mi_tabla.search('edad', 25)
# resultados_rango = mi_tabla.range_search('edad', 25, 35)

## 8. Cargar desde archivo CSV externo

In [ ]:
# Guardar CSV de ejemplo
save_sample_csv('empleados.csv')

# Crear tabla y cargar desde archivo
tabla_archivo = EmployeeTable(bplus_order=5)
tabla_archivo.load_from_file('empleados.csv')

# Crear índices
tabla_archivo.create_index('edad')
tabla_archivo.create_index('cargo')
tabla_archivo.create_index('sueldo')

# Mostrar estadísticas
tabla_archivo.show_index_stats()

## 9. Análisis de Complejidad del Árbol B+

### Complejidad temporal:

| Operación | Sin Índice | Con Árbol B+ |
|-----------|------------|-------------|
| Búsqueda exacta | O(n) | O(log n) |
| Búsqueda por rango | O(n) | O(log n + k) * |
| Inserción | O(1) | O(log n) |
| Recorrido ordenado | O(n log n) | O(n) ** |
| Encontrar mínimo/máximo | O(n) | O(log n) |

\* k = número de resultados  
\*\* usando hojas enlazadas

### Ventajas del Árbol B+ sobre BST:

1. **Siempre balanceado**: Nunca degenera a O(n)
2. **Hojas enlazadas**: Recorrido secuencial O(n) muy eficiente
3. **Alto factor de ramificación**: Árbol más ancho, menos alto
4. **Optimizado para disco**: Menos accesos a disco en bases de datos reales
5. **Datos en hojas**: Nodos internos más pequeños, caben más en memoria

### Ventajas del Árbol B+ sobre Hash Table:

1. **Mantiene orden**: Recorrido ordenado eficiente
2. **Búsqueda por rango**: O(log n + k) vs O(n) en hash
3. **Sin colisiones**: No necesita manejar colisiones
4. **Predecible**: Rendimiento garantizado O(log n)

### Desventajas:

1. **Más complejo**: Implementación más difícil que BST o Hash
2. **Overhead**: Usa más memoria que lista simple
3. **Inserción más lenta**: O(log n) vs O(1) en Hash Table

### ¿Cuándo usar Árbol B+?

- ✅ Bases de datos (es el estándar)
- ✅ Búsquedas por rango frecuentes
- ✅ Necesitas datos ordenados
- ✅ Muchas lecturas, pocas escrituras
- ✅ Almacenamiento en disco

### Parámetros de ajuste:

- **Orden pequeño (3-5)**: Mejor para datos en memoria, fácil de visualizar
- **Orden grande (50-200)**: Mejor para datos en disco, menos accesos

En este notebook usamos orden pequeño (3-5) para que sea fácil ver la estructura.
En bases de datos reales se usa orden mucho mayor (50-200) para minimizar altura.

## 10. Comparación: Árbol B+ vs Otras Estructuras

### Ejemplo: Buscar empleados entre 30 y 40 años

```python
# Con Árbol B+:
# 1. Buscar primer valor (30) → O(log n)
# 2. Seguir enlaces de hojas hasta 40 → O(k)
# Total: O(log n + k)

# Con BST:
# 1. Buscar cada valor del rango → O(log n) por valor
# 2. Recorrer subárbol → O(k)
# Total: O(log n + k) pero más lento en práctica

# Con Hash Table:
# 1. Buscar cada valor individualmente → O(1) * k valores
# 2. Pero necesitas saber qué valores buscar!
# Total: O(k) solo si sabes qué buscar, O(n) si no

# Sin índice:
# 1. Recorrer todos los empleados → O(n)
# Total: O(n)
```

## 11. Conclusiones

### Este notebook demuestra:

1. **Implementación completa de Árbol B+** desde cero
   - Nodos internos y hojas
   - Inserción con división de nodos
   - Hojas enlazadas
   - Balanceo automático

2. **Sistema de base de datos funcional**
   - Tabla de empleados
   - Índices configurables usando B+
   - Búsquedas eficientes
   - Carga desde CSV

3. **Análisis de rendimiento**
   - Comparación con/sin índices
   - Mediciones de tiempo reales
   - Análisis de complejidad

4. **Conceptos de bases de datos reales**
   - CREATE INDEX
   - SELECT con WHERE
   - SELECT con BETWEEN
   - SELECT con ORDER BY

### ¿Por qué Árbol B+ en bases de datos?

Todas las bases de datos principales usan Árbol B+ o variantes:

- **MySQL/InnoDB**: Árbol B+ con factor de ramificación ~1200
- **PostgreSQL**: Árbol B+ llamado "btree"
- **Oracle**: Árbol B+ para índices primarios
- **SQLite**: Árbol B+ con páginas de 4KB
- **MongoDB**: Árbol B para índices

El Árbol B+ es LA estructura fundamental de las bases de datos modernas.

### Experimentos sugeridos:

1. Cambiar el orden del árbol y ver cómo afecta la altura
2. Cargar muchos más datos (1000+) y medir rendimiento
3. Comparar búsqueda exacta vs búsqueda por rango
4. Implementar DELETE y ver cómo se rebalance el árbol
5. Visualizar la estructura del árbol

### Recursos adicionales:

- Database Internals by Alex Petrov
- Introduction to Algorithms (CLRS) - Capítulo sobre Árboles B
- SQLite source code (excelente implementación de B+)
- PostgreSQL documentation sobre btree